# Fourth Hands-On Session
---
- Energy effects: intruducing dispersion
---

The following exercises might not be addressed during the course, but we encourage you to explore them after the course.
They are examples that show how one can extend our simple tools to investigate and better understand more complex systems.

In [ ]:
# Import custom tracking functions and useful libraries
from tracking_library import *

# Set up plotting
params = {
    "legend.fontsize": "x-large",
    "figure.figsize": (15, 5),
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
plt.rcParams.update(params)

## Introducing 3x3 elements

In order to consider energy effects we need to introduce at least $3\times3$ matrices for drift space, quadrupoles, and sector dipoles, where the third coordinate corresponds to the particle energy deviation $\mathrm{d}p/p_0$. Recall [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/):
 
<p align="center">
<img src=./_img_exercises/slide_3x3matrices.png width="60%">
</p>

- **These elements are already provided in our `tracking_library` toolbox:**

In [ ]:
from tracking_library import B3, D3, Q3

help(D3)
help(Q3)
help(B3)

### Exercise 25

Setting the scene: build a beam line of 6 FODO cells (e.g., L_cell = 4 m) with a phase advance of 60 degrees/cell (using 3x3 thin quadrupoles just introduced). 
Add a sector bending magnet (3x3 version) with length 1 m and bending angle $\phi=10$ degrees in the center of each drift. 
At this point, it is not too important where you start your FODO cell.

Verify nothing changes with respect to the same beamline made of 2x2 matrices.

- **Note:** You may have to play with the quadrupole focal length to make the phase advance close to 60 degrees. You should have done this in earlier exercises. 

#### SOLUTION

In [ ]:
# 3x3 case: code here your solution...

In [ ]:
# 2x2 case: code here your solution...

### Exercise 26

Consider the previously defined beamline (3x3 case).
Use the starting conditions $(x_0,x_0',\delta)=(0,0,0)$ and plot the position along the beam line. Repeat this for $\delta=10^{-3}$ and for $\delta=3\times 10^{-3}$. Plot all three traces in the same graph. Discuss what you observe and explain.

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 27
Work out the transverse components of the periodic beam matrix $\sigma_0$. Assume that the geometrical emittance is $\epsilon_0=10^{-6}$ m rad. Furthermore, assume that the momentum spread $\sigma_0(3,3)=\sigma_p^2$ is zero and plot the beam size along the beam line.

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 28
Plot the beam size for for $\sigma_p^2=10^{-3}$ and for $\sigma_p^2=3\times 10^{-3}$. What happens if you change the phase advance of the cell? Try out by slightly changing the focal lengths.

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 29 (guided)

Determine the periodic dispersion at the start of the cell. Then plot the dispersion in the cell.

#### SOLUTION

For that we need to find the *real* periodic solution of the 3x3 one turn matrix

In [ ]:
OTM = getEquivalentElement(beamline)
eigenvalues, eigenvectors = np.linalg.eig(OTM[0]["matrix"])
print(eigenvalues)
print(eigenvectors[:, 2])

Note that the third eigenvalue is real, i.e. is associated to a real eigenvector, which is our solution:

In [ ]:
dispersion = np.real(eigenvectors[:, 2] / eigenvectors[2, 2])
print(dispersion)

Sanity check: one can see that indeeed is a periodic solution!

In [ ]:
OTM[0]["matrix"] @ dispersion

And track it along our lattice:

In [ ]:
# need to make dispersion array compatible with our transportParticles function
dispersion = np.reshape(dispersion, (3, 1))
transported_particles = transportParticles(dispersion, beamline)

plt.plot(transported_particles["s"], transported_particles["x"], "o-b", lw=3)
plt.grid(True)
plt.xlabel("s [m]")
plt.ylabel("Dx [m]", color="b")
plt.tick_params(axis="y", labelcolor="b")

## Well done !!

=> **If you still have time, continue your learning with the following [notebook](./05_4D_Systems.ipynb)**...